In [1]:
url<-"https://raw.githubusercontent.com/WinVector/zmPDSwR/master/Spambase/spamD.tsv"

In [2]:
spamD <- read.table(url,header=T,sep='\t')
head(spamD)

word.freq.make,word.freq.address,word.freq.all,word.freq.3d,word.freq.our,word.freq.over,word.freq.remove,word.freq.internet,word.freq.order,word.freq.mail,⋯,char.freq.lparen,char.freq.lbrack,char.freq.bang,char.freq.dollar,char.freq.hash,capital.run.length.average,capital.run.length.longest,capital.run.length.total,spam,rgroup
0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,⋯,0.000,0,0.778,0.000,0.000,3.756,61,278,spam,52
0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,⋯,0.132,0,0.372,0.180,0.048,5.114,101,1028,spam,91
0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,⋯,0.143,0,0.276,0.184,0.010,9.821,485,2259,spam,49
0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,⋯,0.137,0,0.137,0.000,0.000,3.537,40,191,spam,88
0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,⋯,0.135,0,0.135,0.000,0.000,3.537,40,191,spam,73
0.00,0.00,0.00,0,1.85,0.00,0.00,1.85,0.00,0.00,⋯,0.223,0,0.000,0.000,0.000,3.000,15,54,spam,45


# Listing 5.1 Building and applying a logistic regression spam model

In [3]:
spamTrain <- subset(spamD,spamD$rgroup>=10)
spamTest <- subset(spamD,spamD$rgroup<=10)
spamVars <- setdiff(colnames(spamD),list("rgroup","spam"))

head(spamVars)

[1] "word.freq.make"    "word.freq.address" "word.freq.all"    
[4] "word.freq.3d"      "word.freq.our"     "word.freq.over"

In [4]:
spamFormula <- as.formula(paste('spam=="spam"',paste(spamVars,collapse = ' + '),sep = '~'))
spamFormula

spam == "spam" ~ word.freq.make + word.freq.address + word.freq.all + 
    word.freq.3d + word.freq.our + word.freq.over + word.freq.remove + 
    word.freq.internet + word.freq.order + word.freq.mail + word.freq.receive + 
    word.freq.will + word.freq.people + word.freq.report + word.freq.addresses + 
    word.freq.free + word.freq.business + word.freq.email + word.freq.you + 
    word.freq.credit + word.freq.your + word.freq.font + word.freq.000 + 
    word.freq.money + word.freq.hp + word.freq.hpl + word.freq.george + 
    word.freq.650 + word.freq.lab + word.freq.labs + word.freq.telnet + 
    word.freq.857 + word.freq.data + word.freq.415 + word.freq.85 + 
    word.freq.technology + word.freq.1999 + word.freq.parts + 
    word.freq.pm + word.freq.direct + word.freq.cs + word.freq.meeting + 
    word.freq.original + word.freq.project + word.freq.re + word.freq.edu + 
    word.freq.table + word.freq.conference + char.freq.semi + 
    char.freq.lparen + char.freq.lbrack + char.freq

In [6]:
spamModel <- glm(spamFormula,family = binomial(link="logit"),data=spamTrain)
spamModel

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Call:  glm(formula = spamFormula, family = binomial(link = "logit"), 
    data = spamTrain)

Coefficients:
               (Intercept)              word.freq.make  
                -1.616e+00                  -3.268e-01  
         word.freq.address               word.freq.all  
                -1.546e-01                   1.494e-01  
              word.freq.3d               word.freq.our  
                 2.194e+00                   4.756e-01  
            word.freq.over            word.freq.remove  
                 7.444e-01                   2.339e+00  
        word.freq.internet             word.freq.order  
                 8.005e-01                   6.449e-01  
            word.freq.mail           word.freq.receive  
                 1.044e-01                  -2.830e-01  
            word.freq.will            word.freq.people  
                -1.315e-01                  -1.501e-01  
          word.freq.report         word.freq.addresses  
                 1.371e-01           

In [7]:
spamTrain$pred <- predict(spamModel,newdata = spamTrain,type = "response")
spamTest$pred <- predict(spamModel,newdata = spamTest,type = "response")

print(with(spamTest,table(y=spam,glmPred=pred>0.5)))

          glmPred
y          FALSE TRUE
  non-spam   284   14
  spam        24  174


# Listing 5.2 Spam classifications